# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [19]:
nltk.download(['punkt', 'wordnet','stopwords','averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [34]:
# import libraries
import re
import nltk
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from nltk import pos_tag, ne_chunk
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
from sklearn.ensemble import GradientBoostingClassifier
import pickle

In [20]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponseTable',engine)

In [21]:
pd.set_option('display.max_columns', None) #display all columns
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.766478,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,0.063778,0.111497,0.088267,0.015449,0.023039,0.011367,0.033377,0.045545,0.131446,0.065037,0.045812,0.050847,0.020293,0.006065,0.010795,0.004577,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.423079,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,0.244361,0.314752,0.283688,0.123331,0.150031,0.106011,0.179621,0.208500,0.337894,0.246595,0.209081,0.219689,0.141003,0.077643,0.103338,0.067502,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
X= df['message']
Y= df.iloc[:,4:]

In [23]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [24]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

### 2. Write a tokenization function to process your text data

In [25]:
def tokenize(text):
    
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, text)
    
    # replace each url in text string with placeholder
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    #Remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ",text)
    
    #Remove extra space within a meassage 
    text = " ".join(text.split())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    # remove stop words
    clean_tokens = [w for w in clean_tokens if w not in stopwords.words("english")]
    
    return clean_tokens
    

In [26]:
#test tokenize
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'needs', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
# build our first pipeline using logisticRegression
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moclf', MultiOutputClassifier(LogisticRegression()))    
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
Y_pred = pipeline.predict(X_test)

target_names = Y.columns
print(classification_report(Y_test, Y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.85      0.95      0.90      5058
               request       0.82      0.54      0.65      1144
                 offer       0.00      0.00      0.00        21
           aid_related       0.77      0.66      0.71      2718
          medical_help       0.65      0.14      0.23       521
      medical_products       0.83      0.18      0.30       350
     search_and_rescue       0.80      0.05      0.09       174
              security       0.00      0.00      0.00       120
              military       0.72      0.10      0.18       209
                 water       0.80      0.49      0.61       425
                  food       0.85      0.58      0.69       753
               shelter       0.84      0.44      0.57       568
              clothing       0.86      0.18      0.30        98
                 money       0.67      0.04      0.08       136
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f13cc4a8bf8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('moclf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dt

In [33]:
parameters = {
    'vect__max_df': (0.5, 0.75),
    'moclf__estimator__C': (0.1, 1),
    'moclf__estimator__penalty': ('l1', 'l2')}

cv = GridSearchCV(pipeline, param_grid=parameters,verbose =2, n_jobs=-1) 
#verbose helped to see the progress of our grid searc

In [34]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.5 
[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.5, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.5 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.5, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.75 
[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.75, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.75 
[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.75, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.75 
[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l1, vect__max_df=0.75, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l2, vect__max_df=0.5 
[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l2, vect__max_df=0.5, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator__penalty=l2, vect__max_df=0.5 
[CV]  moclf__estimator__C=0.1, moclf__estimator__penalty=l2, vect__max_df=0.5, total= 1.4min
[CV] moclf__estimator__C=0.1, moclf__estimator

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 56.8min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75), 'moclf__estimator__C': (0.1, 1), 'moclf__estimator__penalty': ('l1', 'l2')},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [36]:
y_pred = cv.predict(X_test)

In [35]:
cv.best_params_

{'moclf__estimator__C': 1,
 'moclf__estimator__penalty': 'l1',
 'vect__max_df': 0.5}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [37]:
target_names = Y.columns
print(classification_report(Y_test, Y_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.85      0.95      0.90      5058
               request       0.82      0.54      0.65      1144
                 offer       0.00      0.00      0.00        21
           aid_related       0.77      0.66      0.71      2718
          medical_help       0.65      0.14      0.23       521
      medical_products       0.83      0.18      0.30       350
     search_and_rescue       0.80      0.05      0.09       174
              security       0.00      0.00      0.00       120
              military       0.72      0.10      0.18       209
                 water       0.80      0.49      0.61       425
                  food       0.85      0.58      0.69       753
               shelter       0.84      0.44      0.57       568
              clothing       0.86      0.18      0.30        98
                 money       0.67      0.04      0.08       136
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
           n_jobs=1))])

##### the model performs the same after optimization, therefore below we find that AdaBoostClassifier performs better than logistic regression, which we will use along with adding a verb extractor feature to our pipeline:

In [26]:
#building a pipeline using AdaBoostClassifier performs better than LogisticRegression
pipeline_3 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('moclf', MultiOutputClassifier(AdaBoostClassifier()))    
    ])

In [27]:
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X, Y)
pipeline_3.fit(X3_train, Y3_train)
Y3_pred = pipeline_3.predict(X3_test)
#def display_results(y_test, y_pred):
target_names = Y.columns
print(classification_report(Y3_test, Y3_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      5012
               request       0.76      0.50      0.61      1157
                 offer       0.00      0.00      0.00        30
           aid_related       0.77      0.60      0.67      2687
          medical_help       0.65      0.25      0.36       505
      medical_products       0.62      0.30      0.40       342
     search_and_rescue       0.55      0.16      0.25       182
              security       0.19      0.04      0.07       119
              military       0.60      0.35      0.44       209
                 water       0.77      0.65      0.71       426
                  food       0.83      0.64      0.73       721
               shelter       0.76      0.52      0.62       573
              clothing       0.65      0.37      0.47       110
                 money       0.54      0.25      0.34       142
        missing_people       0.53      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [27]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class, as in the course material, extracts the starting verb of a sentence
    """
    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        #attach POS tags to sentences 
        #try/except used because pos_tags[0] raises out of range exception although our data has no nulls)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except: pass
        return False
                       
                
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

    
    
pipeline_boost = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('moclf', MultiOutputClassifier(AdaBoostClassifier()))
    ])


In [28]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X, Y)

In [18]:
pipeline_boost.fit(X2_train, Y2_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [145]:
Y2_pred = pipeline_boost.predict(X2_test)

target_names = Y.columns
print(classification_report(Y2_test, Y2_pred, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      5003
               request       0.76      0.50      0.60      1118
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.60      0.67      2708
          medical_help       0.63      0.23      0.34       569
      medical_products       0.70      0.32      0.44       347
     search_and_rescue       0.57      0.15      0.23       192
              security       0.47      0.05      0.10       130
              military       0.64      0.33      0.44       202
                 water       0.73      0.62      0.67       401
                  food       0.82      0.65      0.72       729
               shelter       0.78      0.54      0.64       574
              clothing       0.64      0.33      0.44       109
                 money       0.56      0.25      0.34       167
        missing_people       0.38      

In [160]:
pipeline_boost.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('moclf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
  

In [29]:
parameters = {
    'moclf__estimator__learning_rate' : [0.05,0.1,1],
    'moclf__estimator__n_estimators': [50, 100]}
    #'moclf__estimator__base_estimator':[AdaBoostClassifier(), GradientBoostingClassifier()],
    #'moclf__estimator__algorithm': ['SAMME.R', 'SAMM']}

cv2 = GridSearchCV(pipeline_boost, param_grid=parameters,verbose =2, n_jobs=-1) 
#verbose helped to see the progress of our grid search

In [30]:
cv2.fit(X2_train, Y2_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=50 
[CV]  moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=50, total= 4.3min
[CV] moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  6.6min remaining:    0.0s


[CV]  moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=50, total= 4.3min
[CV] moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=50 
[CV]  moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=50, total= 4.3min
[CV] moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=100 
[CV]  moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=100, total= 5.4min
[CV] moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=100 
[CV]  moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=100, total= 5.4min
[CV] moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=100 
[CV]  moclf__estimator__learning_rate=0.05, moclf__estimator__n_estimators=100, total= 5.4min
[CV] moclf__estimator__learning_rate=0.1, moclf__estimator__n_estimators=50 
[CV]  moclf__estimator__learning_rate=0.1, moclf__estimator__n_estimators=50, total= 4.3min
[CV] moclf__estimator__learning_rate=0.1, moclf_

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 128.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'moclf__estimator__learning_rate': [0.05, 0.1, 1], 'moclf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [39]:
cv2.best_params_

{'moclf__estimator__learning_rate': 1, 'moclf__estimator__n_estimators': 100}

### 9. Export your model as a pickle file

In [35]:
pickle.dump(cv2, open('classifier.pkl', 'wb'))

##### Testing performance of our final model:

In [37]:
loaded_model = pickle.load(open('classifier.pkl', 'rb'))
#count_vect = pickle.load(open('count_vect', 'rb'))
result = loaded_model.predict(X2_test)

In [42]:
target_names = Y.columns
print(classification_report(Y2_test, result, target_names=target_names))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.87      4983
               request       0.73      0.54      0.62      1103
                 offer       0.00      0.00      0.00        28
           aid_related       0.77      0.63      0.69      2756
          medical_help       0.56      0.24      0.34       515
      medical_products       0.59      0.32      0.42       321
     search_and_rescue       0.56      0.20      0.29       203
              security       0.22      0.03      0.06       116
              military       0.59      0.35      0.44       226
                 water       0.71      0.67      0.69       424
                  food       0.80      0.67      0.73       749
               shelter       0.73      0.52      0.61       578
              clothing       0.66      0.43      0.52       101
                 money       0.54      0.24      0.33       160
        missing_people       0.45      

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.